In [ ]:

import glob
import os
import sys
import time
import argparse
import random
import carla

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


In [ ]:

def main():
    client = carla.Client('127.0.0.1', 2000)
    client.set_timeout(10.0)
    world = client.get_world()

    # Move the spectator camera
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(carla.Location(x=0, y=0, z=50), carla.Rotation(pitch=-90)))

    # Setup Traffic Manager
    traffic_manager = client.get_trafficmanager(6000)
    traffic_manager.set_global_distance_to_leading_vehicle(2.5)
    time.sleep(2)

    # Enable synchronous mode
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 0.05
    world.apply_settings(settings)

    # Retrieve a vehicle blueprint
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[0]

    # Define spoofed GPS coordinates
    spoofed_latitude = 52.0
    spoofed_longitude = 4.0

    # Spawn vehicles and attach GPS sensors
    num_vehicles = 2
    vehicles = []
    gps_sensors = []
    spawn_location = carla.Location(x=0, y=0, z=1)

    for i in range(num_vehicles):
        spawn_point = carla.Transform(
            location=carla.Location(
                x=spawn_location.x + i * 10,
                y=spawn_location.y,
                z=spawn_location.z
            ),
            rotation=carla.Rotation(yaw=0)
        )

        vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
        if vehicle:
            vehicle.set_autopilot(True, traffic_manager.get_port())
            vehicles.append(vehicle)
            print(f"Vehicle {i} successfully spawned at {spawn_point.location}")

            # Attach GPS sensor
            gps_bp = blueprint_library.find('sensor.other.gnss')
            gps_transform = carla.Transform(carla.Location(x=1.0, z=2.8))
            gps_sensor = world.spawn_actor(gps_bp, gps_transform, attach_to=vehicle)

            def gps_callback(event, vehicle_id=vehicle.id):
                original_lat = event.latitude
                original_long = event.longitude
                print(f"[Vehicle {vehicle_id}] Original GPS: Latitude={original_lat}, Longitude={original_long}")
                print(f"[Vehicle {vehicle_id}] Spoofed GPS: Latitude={spoofed_latitude}, Longitude={spoofed_longitude}")

            gps_sensor.listen(lambda event, id=vehicle.id: gps_callback(event, id))
            gps_sensors.append(gps_sensor)
        else:
            print("Spawn failed.")

    return world, gps_sensors, vehicles, settings


In [ ]:

world, gps_sensors, vehicles, settings = main()

try:
    while True:
        world.tick()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    print("Cleaning up...")
    for gps_sensor in gps_sensors:
        gps_sensor.stop()
        gps_sensor.destroy()
    for vehicle in vehicles:
        vehicle.destroy()
    settings.synchronous_mode = False
    world.apply_settings(settings)
    print("Done.")
